In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
# import modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split


%matplotlib inline

In [3]:
# load dataset

df = pd.read_csv('wine.csv')

In [4]:
df

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df[['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]
y = df['quality']

In [5]:
# Creating a train/test split
train_test_split_fraction = 0.80
split_index = int(df.shape[0] * train_test_split_fraction)
df_train = df[:split_index]
df_test = df[split_index:]

target = pd.get_dummies(df['quality']).values
target[:5]

array([[0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0]], dtype=uint8)

In [6]:
# Slecting the featues and the target
X_train = df_train.drop('quality', axis=1).values
X_test = df_test.drop('quality', axis=1).values

y_train = target[:split_index]
y_test = target[split_index:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5197, 12)
(1300, 12)
(5197, 7)
(1300, 7)


In [7]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [8]:
X_train[:5]

tensor([[0.0000e+00, 7.4000e+00, 7.0000e-01, 0.0000e+00, 1.9000e+00, 7.6000e-02,
         1.1000e+01, 3.4000e+01, 9.9780e-01, 3.5100e+00, 5.6000e-01, 9.4000e+00],
        [1.0000e+00, 7.8000e+00, 8.8000e-01, 0.0000e+00, 2.6000e+00, 9.8000e-02,
         2.5000e+01, 6.7000e+01, 9.9680e-01, 3.2000e+00, 6.8000e-01, 9.8000e+00],
        [2.0000e+00, 7.8000e+00, 7.6000e-01, 4.0000e-02, 2.3000e+00, 9.2000e-02,
         1.5000e+01, 5.4000e+01, 9.9700e-01, 3.2600e+00, 6.5000e-01, 9.8000e+00],
        [3.0000e+00, 1.1200e+01, 2.8000e-01, 5.6000e-01, 1.9000e+00, 7.5000e-02,
         1.7000e+01, 6.0000e+01, 9.9800e-01, 3.1600e+00, 5.8000e-01, 9.8000e+00],
        [4.0000e+00, 7.4000e+00, 7.0000e-01, 0.0000e+00, 1.9000e+00, 7.6000e-02,
         1.1000e+01, 3.4000e+01, 9.9780e-01, 3.5100e+00, 5.6000e-01, 9.4000e+00]],
       dtype=torch.float64)

In [9]:
y_train[:5]

tensor([[0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [10]:
nb_hidden_neurons = 10
nb_classes = len(pd.unique(df['quality']))
nb_classes

7

In [ ]:
# Source
# https://janakiev.com/blog/pytorch-iris/
